# 상관 관계 기능 섭동을 통한 LinearExplainer의 수학

 `LinearExplainer(model, prior, feature_perturbation="correlation_dependent")`를 쓸 때, 의 경우, $E[f(x) \mid do(X_S = x_S)]$를 사용하여 특징 집합 $S$의 영향을 측정하는 것이 아니라, 입력 특징을 나타내는 무작위 변수 $X$가 다변량 가우시안 분포를 따른다는 가정 하에 $E[f(x) \mid X_S = x_s]$를 사용합니다. 이러한 방식으로 SHAP 값을 계산하려면 모든 특징 하위 집합에 대해 다변량 가우스 분포 하에서 조건부 기대치를 계산해야 합니다. 이렇게 하면 기하급수적인 수의 용어에 대해 많은 행렬을 일치시켜야 하므로 몇 개 이상의 특징을 가진 모델에서는 다루기 어렵습니다.

이 문서에서는 한 번만 수행한 다음 원하는 만큼의 샘플에 적용할 수 있는 샘플링 절차를 사용하여 필요한 모든 선형 대수를 미리 계산하는 데 사용한 수학을 간략하게 설명합니다. 이렇게 하면 무차별 대입 방식에 비해 계산 속도가 크게 빨라집니다. 이러한 모든 계산은 선형 모델 $f(x) = \beta x$를 설명한다는 사실에 의존한다는 점에 유의하세요.

대부분의 설명자가 사용하는 개입 형식의 SHAP 값의 순열 정의는 다음과 같습니다.

$$
\phi_i = \frac{1}{M!} \sum_R E[f(X) \mid do(X_{S_i^R \cup i} = x_{S_i^R \cup i})] - E[f(X) \mid do(X_{S_i^R} = x_{S_i^R})]
$$

하지만 여기서는 비개입 조건부 기대 형식을 사용하겠습니다. 여기서 무작위 변수 $X$에 대한 명시적 참조를 삭제하여 표기법을 단순화했습니다.).

$$
\phi_i = \frac{1}{M!} \sum_R E[f(x) \mid x_{S_i^R \cup i}] - E[f(x) \mid x_{S_i^R}]
$$

여기서 $f(x) = \beta x + b$이고, $\beta$는 행 벡터, $b$는 스칼라입니다.

f(x)를 선형 함수 정의로 바꾸면 다음과 같은 결과가 나옵니다:

\begin{align}
\phi_i = \frac{1}{M!} \sum_R E[\beta x + b \mid x_{S_i^R \cup i}] - E[\beta x + b \mid x_{S_i^R}] \\
 = \beta \frac{1}{M!} \sum_R E[x \mid x_{S_i^R \cup i}] - E[x \mid x_{S_i^R}]
\end{align}

입력 $x$가 평균 $\mu$와 공분산 $\Sigma$를 갖는 다변량 정규 분포를 따른다고 가정합니다. 집합 $S$를 선택하는 투영 행렬을 $P_S$로 나타내면 다음과 같이 됩니다:

\begin{align}
E[x \mid x_S] = [P_{\bar S} \mu + P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} ( P_S x - P_S \mu)] P_{\bar S} + x P_S^T P_S \\
= [P_{\bar S} \mu + P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S (x - \mu)] P_{\bar S} + x P_S^T P_S \\
= [\mu + \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S (x - \mu)] P_{\bar S}^T P_{\bar S} + x P_S^T P_S \\
= P_{\bar S}^T P_{\bar S} [\mu + \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S (x - \mu)] + P_S^T P_S x \\
= P_{\bar S}^T P_{\bar S} \mu + P_{\bar S}^T P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S x - P_{\bar S}^T P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S \mu + P_S^T P_S x \\
= [P_{\bar S}^T P_{\bar S} - P_{\bar S}^T P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S] \mu + [P_S^T P_S + P_{\bar S}^T P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S] x
\end{align}

만약 $R_S = P_{\bar S}^T P_{\bar S} \Sigma P_S^T (P_S \Sigma P_S^T)^{-1} P_S$ 이고 $Q_S = P_S^T P_S$ 라면, 이렇게 할 수 있다.

\begin{align}
E[x \mid x_S] = [Q_{\bar S} - R_S] \mu + [Q_S + R_S] x
\end{align}

혹은

\begin{align}
E[x \mid x_{S_i^R \cup i}] = [Q_{\bar{S_i^R \cup i}} - R_{S_i^R \cup i}] \mu + [Q_{S_i^R \cup i} + R_{S_i^R \cup i}] x
\end{align}

의 샤플리 방정식으로 이어집니다.

\begin{align}
\phi_i = \beta \frac{1}{M!} \sum_R  [Q_{\bar{S_i^R \cup i}} - R_{S_i^R \cup i}] \mu + [Q_{S_i^R \cup i} + R_{S_i^R \cup i}] x - [Q_{\bar{S_i^R}} - R_{S_i^R}] \mu - [Q_{S_i^R} + R_{S_i^R}] x \\
= \beta \frac{1}{M!} \sum_R  ([Q_{\bar{S_i^R \cup i}} - R_{S_i^R \cup i}] - [Q_{\bar{S_i^R}} - R_{S_i^R}]) \mu + ([Q_{S_i^R \cup i} + R_{S_i^R \cup i}] - [Q_{S_i^R} + R_{S_i^R}]) x \\
= \beta \left [ \frac{1}{M!} \sum_R  ([Q_{\bar{S_i^R \cup i}} - R_{S_i^R \cup i}] - [Q_{\bar{S_i^R}} - R_{S_i^R}]) \right ] \mu + \beta \left [ \frac{1}{M!} \sum_R ([Q_{S_i^R \cup i} + R_{S_i^R \cup i}] - [Q_{S_i^R} + R_{S_i^R}]) \right ] x
\end{align}



$$
\phi = \beta T x
$$

즉, 무작위 순열 $R$을 여러 번 뽑고 그 결과의 평균을 구하여 변환 행렬 $T$를 미리 계산할 수 있습니다. 일단 $T$를 계산하고 나면 행렬 곱셈을 사용하여 원하는 수의 샘플(또는 해당 모델)을 설명할 수 있습니다.